In [34]:


import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'



In [35]:
!ls

 Clean_Tweets.ipynb  'offenseval-training-v1 (copy).tsv'
 corpus		      offenseval-training-v1.tsv
 glove.6B	      R8


In [36]:
df = pd.read_csv("offenseval-training-v1.tsv", sep ='\t')

In [37]:
df.head()


,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [38]:
df.rename({'tweet':'text', 'subtask_a':'sentiment', 'subtask_b':'subb', 'subtask_c':'subc'}, axis=1,inplace=True)


In [39]:
df.head()

,id,text,sentiment,subb,subc
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [40]:
#z = {'NOT' : 0 ,'OFF' : 1}
#df['sentiment'] = df.sentiment.map(z)

In [41]:
'''NOT -- 0 OFF -- 1'''

'NOT -- 0 OFF -- 1'

In [42]:
df.head()

,id,text,sentiment,subb,subc
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [43]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 5 columns):
id           2499 non-null int64
text         2499 non-null object
sentiment    2499 non-null object
subb         838 non-null object
subc         750 non-null object
dtypes: int64(1), object(4)
memory usage: 97.7+ KB


In [44]:


df.sentiment.value_counts()



NOT    1661
OFF     838
Name: sentiment, dtype: int64

In [45]:
df.drop(['subb','subc'],axis=1,inplace=True)

In [46]:
df.head()

,id,text,sentiment
0,86426,@USER She should ask a few native Americans wh...,OFF
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,16820,Amazon is investigating Chinese employees who ...,NOT
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT


In [47]:


from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()



In [48]:


import re

from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
url = r'URL'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner(text):
    
    try:
        bom_removed = text.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = text
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(url, '', stripped)
    
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()



In [49]:
testing = df.text[:]

In [50]:


test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))



In [51]:
test_result

['she should ask few native americans what their take on this is',
 'go home you re drunk maga trump',
 'amazon is investigating chinese employees who are selling internal data to third party sellers looking for an edge in the competitive marketplace amazon maga kag china tcot',
 'someone should vetaken this piece of shit to volcano',
 'obama wanted liberals amp illegals to move into red states',
 'liberals are all kookoo',
 'oh noes tough shit',
 'was literally just talking about this lol all mass shootings like that have been set ups it propaganda used to divide us on major issues like gun control and terrorism',
 'buy more icecream',
 'canada doesn need another cuck we already have enough looneyleft liberals king up our great country qproofs trudeaumustgo',
 'it not my fault you support gun control',
 'what the difference between kavanaugh and one of these men admitted to groping year old girl years ago the other is going to be confirmed to the scj demsarefrauds demsaredone walkaway

In [52]:
%%time
#NOT REQD.
print ("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(0,13240):
    if( (i+1)%10000 == 0 ):
        print ("Tweets has been processed")                                                                    
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...



KeyError: 2499

In [53]:
len(test_result)

2499

In [54]:
df2=df[df.sentiment == 'OFF']

df2.head()
df2.to_csv('offensive.csv',encoding='utf-8',header=None, index= None)

In [55]:
df3=df[df.sentiment == 'NOT']

df3.head()
df3.to_csv('not_offensive.csv',encoding='utf-8',header=None, index= None)

In [56]:
df4=df.iloc[:, ::-1]

df4.head()
#df4.to_csv('train.txt',encoding='utf-8',header=None, index= None)

,sentiment,text,id
0,OFF,@USER She should ask a few native Americans wh...,86426
1,OFF,@USER @USER Go home you’re drunk!!! @USER #MAG...,90194
2,NOT,Amazon is investigating Chinese employees who ...,16820
3,OFF,"@USER Someone should'veTaken"" this piece of sh...",62688
4,NOT,@USER @USER Obama wanted liberals &amp; illega...,43605


In [57]:
df4.to_csv('train.txt',encoding='utf-8',header=None, index= None)

In [58]:
df5=df4.text

df5.head()
df5.to_csv('train_all.txt',encoding='utf-8',header=None, index= None)

In [59]:
""""df6=df4.id

df6.head()
df6.to_csv('train_index.txt',encoding='utf-8',header=None, index= None)"""

'"df6=df4.id\n\ndf6.head()\ndf6.to_csv(\'train_index.txt\',encoding=\'utf-8\',header=None, index= None)'

In [61]:
ndf = pd.read_csv("testset-taska.tsv", sep =',')

In [62]:
ndf.head()

,Unnamed: 0,id,text,sentiment
0,0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,-1
1,1,27014,"#ConstitutionDay is revered by Conservatives, ...",-1
2,2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,-1
3,3,13876,#Watching #Boomer getting the news that she is...,-1
4,4,60133,#NoPasaran: Unity demo to oppose the far-right...,-1


In [63]:
z = {-1 :'NOT'}
ndf['sentiment'] = ndf.sentiment.map(z)

In [64]:
ndf.head()

,Unnamed: 0,id,text,sentiment
0,0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,NOT
1,1,27014,"#ConstitutionDay is revered by Conservatives, ...",NOT
2,2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
3,3,13876,#Watching #Boomer getting the news that she is...,NOT
4,4,60133,#NoPasaran: Unity demo to oppose the far-right...,NOT


In [65]:
ndf.drop(['Unnamed: 0'],axis=1,inplace=True)

In [66]:
ndf.head()

,id,text,sentiment
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,NOT
1,27014,"#ConstitutionDay is revered by Conservatives, ...",NOT
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
3,13876,#Watching #Boomer getting the news that she is...,NOT
4,60133,#NoPasaran: Unity demo to oppose the far-right...,NOT


In [67]:
ndf=ndf.iloc[:, ::-1]


ndf.to_csv('test.txt',encoding='utf-8',header=None, index= None)